Leer y procesar imágenes para entrenar:

In [7]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import tensorflow as tf
import matplotlib as plt
from keras import layers, models
import os
import cv2

# tamaño de imagen y epocas del entrenamiento
width = 100
height = 100
epochs = 20

ruta_train = '/content/drive/My Drive/chest_xray/train/'

train_x = []
train_y = []

# leer y procesar imagenes
for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train + i + '/' + j, cv2.IMREAD_GRAYSCALE)  # leer imagen en escala de grises
        if img is None:
            continue

        resized = cv2.resize(img, (width, height))
        resized = resized / 255.0  # normalizar a [0, 1]

        train_x.append(resized)

        if i == "NORMAL":
            train_y.append([0, 1])
        else:
            train_y.append([1, 0])

x_data = np.array(train_x)
y_data = np.array(train_y)

# añadir una dimensión extra para que coincida con la entrada del modelo (capa Conv2D espera (width, height, 1))
x_data = np.expand_dims(x_data, axis=-1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Definición y entrenamiento del modelo:

In [8]:
model = tf.keras.Sequential([
    layers.Conv2D(1000, (3, 3), input_shape=(width, height, 1), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_data, y_data, epochs=epochs)

model.save('aprendePorfavor.keras')

Epoch 1/20
84/84 [==============================] - 113s 1s/step - loss: 0.5609 - accuracy: 0.6928
Epoch 2/20
84/84 [==============================] - 114s 1s/step - loss: 0.3280 - accuracy: 0.8635
Epoch 3/20
84/84 [==============================] - 114s 1s/step - loss: 0.2533 - accuracy: 0.8982
Epoch 4/20
84/84 [==============================] - 114s 1s/step - loss: 0.2270 - accuracy: 0.9165
Epoch 5/20
84/84 [==============================] - 115s 1s/step - loss: 0.2554 - accuracy: 0.9031
Epoch 6/20
84/84 [==============================] - 115s 1s/step - loss: 0.2449 - accuracy: 0.9016
Epoch 7/20
84/84 [==============================] - 115s 1s/step - loss: 0.1862 - accuracy: 0.9217
Epoch 8/20
84/84 [==============================] - 115s 1s/step - loss: 0.2147 - accuracy: 0.9060
Epoch 9/20
84/84 [==============================] - 115s 1s/step - loss: 0.1651 - accuracy: 0.9396
Epoch 10/20
84/84 [==============================] - 115s 1s/step - loss: 0.1572 - accuracy: 0.9366
Epoch 11/

Cargar y procesar imágenes para evaluar el modelo:

In [10]:
import numpy as np
import tensorflow as tf
from keras import layers, models
from google.colab import drive
import os
import cv2

#tamaño de imagen
width = 100
height = 100

# ruta hacia las imagenes para testear
ruta_test = '/content/drive/My Drive/chest_xray/test/'

test_x = []
test_y = []

# cargar el modelo
model = models.load_model('aprendePorfavor.keras')

# leer y procesar imagenes
for i in os.listdir(ruta_test):
    for j in os.listdir(ruta_test + i):
        img = cv2.imread(ruta_test + i + '/' + j, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        resized = cv2.resize(img, (width, height))
        resized = resized / 255.0

        test_x.append(resized)

        if i == "NORMAL":
            test_y.append([0, 1])
        else:
            test_y.append([1, 0])

x_test = np.array(test_x)
y_test = np.array(test_y)

x_test = np.expand_dims(x_test, axis=-1)

Evaluar el modelo:

In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

20/20 [==============================] - 7s 350ms/step - loss: 0.8880 - accuracy: 0.7917
Test accuracy: 0.7916666865348816
